### Building a Machine Learning Classifier: Random Forest Model

+ Random Forest constructs a collection of decision trees and then aggregates the prediction of each tree to determine the final prediction. 

+ Random Forests use **bagging**- Each tree randomly samples from the dataset with replacement resulting in different trees. This is because trees are sensitive to the data they are trained on.

+ Random Forest use **Feature randomness** - Each tree will use different features to make predictions.

+ The idea is to use **bagging** and **feature randomness** for each individual tree to create an uncorrelated forest of trees where prediction by a group of trees is more accurate than any individual tree. 

    * Falls under ensemble methods - Technique that creates multiple models and then combines them to produce better results than any of the single models individually.

    * Used for classification or regression

    * Easily handles outlier, missing values etc

    * Accepts various types of inputs (continuous, ordinal etc)

    * Less likely to overfit

    * Outputs feature importance

In [3]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Random Forest Classifier 

In [4]:
from sklearn.ensemble import RandomForestClassifier
 # feature_importances_ - outputs the value of each feature to the model
 # fit - fit the model and store it as an object
 # predict - use the fit model to predict on the test set
print(dir(RandomForestClassifier))
 # max_depth - how deep each decision tree is, n_estimators - how many decision trees to build within a forest. 
 # Build n decision trees of unlimited depth. Which then would be a vote 
print(help(RandomForestClassifier()))

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
Help on RandomForestClassifier in module sklearn.ensemble._forest object:

class RandomFore

#### KFold - Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout in each iteration

In [5]:
from sklearn.model_selection import KFold, cross_val_score

# n_jobs = -1, building individual trees in parallel
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)

# X_features = (5567, 8106), data['label'] = (5567,)
cross_val_score(rf,X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97845601, 0.97845601, 0.97394429, 0.96316262, 0.97214735])

In [6]:
from sklearn.model_selection import KFold, cross_val_score

help(cross_val_score)

Help on function cross_val_score in module sklearn.model_selection._validation:

cross_val_score(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score=nan)
    Evaluate a score by cross-validation
    
    Read more in the :ref:`User Guide <cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitting the dataset into
        train/test set. Only used in conjunction with a "Group" :term:`cv`

#### Exploring with Test set

In [7]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [8]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train,y_train)

In [9]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse = True)[0:10]

[(0.059449728703890994, 'body_len'),
 (0.05782279546170727, 7350),
 (0.048345498412298696, 3134),
 (0.03413835725990599, 1803),
 (0.032779352085660456, 4796),
 (0.02556302859899682, 1361),
 (0.018977140396974596, 5988),
 (0.018559698132477688, 6285),
 (0.01751343656050382, 2171),
 (0.015220657060487171, 2031)]

In [10]:
y_pred = rf_model.predict(X_test)
# pos_label - is what we want to predict
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

#### Evaluation metrics

+ Accuracy: # predicted correctly / # total observations
+ Precision: # predicted as spam that are actually spam / # total predicted as spam
+ Recall: # predicted as spam that are actually spam / # total that are actually spam

In [11]:
print("Precision: {} \ Recall: {} \ Accuracy: {}".format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred == y_test).sum() / len(y_pred), 3)))

Precision: 1.0 \ Recall: 0.631 \ Accuracy: 0.948


In [12]:
# 100% precision: When the model identified a msg as spam it actually was spam 100% of the time
# Recall: only 56.1% of all the emails was actually placed in the spam folder
# Accuracy: Of all the email whether spam or not spam, 93.9% of the time they were identified correctly. 

In [14]:
print(support)

None


#### Exploring Random Forest with grid-search

Grid-search - Defining a grid with different hyperparameter settings and exploring a model.fit with each combining of hyperparameters

In [15]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [16]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth,n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test,y_pred, pos_label='spam', average='binary')
    print("Est: {} \ Depth: {} \ Precision: {} \ Recall: {} \ Accuracy: {}".format(n_est, depth, round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred == y_test).sum() / len(y_pred), 3)))

In [17]:
for n_est in [10,50,100]:
    for depth in [10,20,30,None]: # None correlates to depth = 100
        train_RF(n_est,depth) 
# As we can see Depth plays a huge role in affecting recall.         

Est: 10 \ Depth: 10 \ Precision: 1.0 \ Recall: 0.385 \ Accuracy: 0.921
Est: 10 \ Depth: 20 \ Precision: 1.0 \ Recall: 0.657 \ Accuracy: 0.956
Est: 10 \ Depth: 30 \ Precision: 1.0 \ Recall: 0.748 \ Accuracy: 0.968
Est: 10 \ Depth: None \ Precision: 1.0 \ Recall: 0.811 \ Accuracy: 0.976
Est: 50 \ Depth: 10 \ Precision: 1.0 \ Recall: 0.238 \ Accuracy: 0.902
Est: 50 \ Depth: 20 \ Precision: 1.0 \ Recall: 0.601 \ Accuracy: 0.949
Est: 50 \ Depth: 30 \ Precision: 1.0 \ Recall: 0.762 \ Accuracy: 0.969
Est: 50 \ Depth: None \ Precision: 1.0 \ Recall: 0.853 \ Accuracy: 0.981
Est: 100 \ Depth: 10 \ Precision: 1.0 \ Recall: 0.238 \ Accuracy: 0.902
Est: 100 \ Depth: 20 \ Precision: 1.0 \ Recall: 0.664 \ Accuracy: 0.957
Est: 100 \ Depth: 30 \ Precision: 1.0 \ Recall: 0.755 \ Accuracy: 0.969
Est: 100 \ Depth: None \ Precision: 0.992 \ Recall: 0.853 \ Accuracy: 0.98


#### Exploring RandomForest with GridSearch and Cross Validation

In [18]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [20]:
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300], 'max_depth': [30, 60, 90, None]}
# scikit-learn object that needs to be fitted with the training data across 5 different folds with the different parameter settings
gs = GridSearchCV(rf, param, cv=5,  n_jobs=-1 )
gs_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5] # printing out top 5 models

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,24.323998,0.291568,0.301624,0.030643,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.978456,0.979354,0.973944,0.969452,0.970350,0.974311,0.004051,1
11,43.466763,4.506494,0.328161,0.026728,None,300,"{'max_depth': None, 'n_estimators': 300}",0.979354,0.978456,0.975741,0.969452,0.968553,0.974311,0.004504,1
7,26.969908,0.520698,0.362441,0.040569,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.973968,0.978456,0.974843,0.971249,0.971249,0.973953,0.002672,3
6,3.188771,0.044501,0.228894,0.033038,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.979354,0.976661,0.972147,0.969452,0.972147,0.973952,0.003558,4
8,51.643735,1.619447,0.377994,0.055302,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.973968,0.974843,0.968553,0.971249,0.973234,0.003088,5


In [21]:
rf = RandomForestClassifier()
param = {'n_estimators': [10,150,300], 'max_depth': [30, 60, 90, None]}
# scikit-learn object that needs to be fitted with the training data across 5 different folds with the different parameter settings
gs = GridSearchCV(rf, param, cv=5,  n_jobs=-1 )
gs_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5] # printing out top 5 models

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,47.335807,5.376570,0.315613,0.029703,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977558,0.974865,0.973944,0.968553,0.970350,0.973054,0.003223,1
10,33.079222,0.435513,0.327575,0.010780,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977558,0.974865,0.972147,0.966757,0.973046,0.972875,0.003575,2
3,3.043112,0.264954,0.191472,0.008200,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.976661,0.977558,0.973944,0.970350,0.965858,0.972874,0.004313,3
7,28.490752,0.238614,0.343214,0.019155,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.976661,0.973070,0.973944,0.967655,0.970350,0.972336,0.003088,4
8,55.958418,1.509979,0.428870,0.073132,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.977558,0.973070,0.973046,0.966757,0.970350,0.972156,0.003556,5
